In [9]:
import json
from langchain.chains.summarize import load_summarize_chain
from langchain.docstore.document import Document
from langchain.prompts import PromptTemplate
from langchain_groq import ChatGroq

In [10]:
llm = ChatGroq(model_name = "llama3-8b-8192", groq_api_key = "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")

In [11]:
with open('combined_summary_by_title.json', 'r') as file:
    documents = json.load(file)
    
documents

{'FP Country Execution Plan - NIGERIA_Strategy Execution Plan_10012014': 'Title - FP Country Execution Plan - NIGERIA_Strategy Execution Plan_10012014\nAbstract - An execution plan for how the Gates Foundation plans to work in the Nigeria to contribute to the FP2020 goal of 120 million women by 2020\nSummary - The image is a presentation slide from the Bill & Melinda Gates Foundation, specifically related to Nigeria\'s Family Planning Strategy Execution Plan for the fourth quarter of 2014. The slide is minimalistic, with a white background and a maroon header that includes the foundation\'s logo and name. Below this, in bold black font, are the words "FAMILY PLANNING STRATEGY EXECUTION" followed by "Nigeria Country Execution Plan." The date "Q4 2014" is also present, indicating the time frame for this plan. The footer contains a copyright notice for the year 2014 and the name of the foundation. The overall design suggests a formal and professional context, likely used in a business or 

In [12]:
map_prompt_template = """
                      Write a detailed and elaborated summary of the following text that includes the main points and any important details.
                      Aim for a summary length of approximately 1500 words
                      {text}
                      """

map_prompt = PromptTemplate(template=map_prompt_template, input_variables=["text"])

In [13]:

combine_prompt_template = """
                      Write a comprehensive summary of the following text delimited by triple backquotes.
                      Aim for a summary length of approximately 800 words with out missing the important information the text.
                      ```{text}```
                      COMPREHENSIVE SUMMARY:
                      """

combine_prompt = PromptTemplate(template=combine_prompt_template, input_variables=["text"])

In [14]:
stuff_prompt_template = """
                      Write a detailed and elaborated summary of the following text that includes the main points and any important details.
                      Aim for a summary length of approximately 1500 words
                      {text}
                      """
                    
stuff_prompt = PromptTemplate(template=stuff_prompt_template, input_variables=["text"])

In [15]:
# summary_chain = load_summarize_chain(llm, chain_type = "stuff", verbose = True,  prompt = stuff_prompt )

In [21]:
summary_chain = load_summarize_chain(llm, chain_type = "map_reduce", verbose = True,  map_prompt=map_prompt, combine_prompt=combine_prompt, return_intermediate_steps=True, )

In [22]:
summarized_docs = {}

In [23]:
docs = []
for title, content in documents.items():
    print(f"summarising document with title: {title}" )

    doc =Document(page_content = content)

    summary_result = summary_chain.invoke([doc])
    
    summary = summary_result['output_text']
    
    summarized_docs[title] = summary    

summarising document with title: FP Country Execution Plan - NIGERIA_Strategy Execution Plan_10012014


> Entering new MapReduceDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

                      Write a detailed and elaborated summary of the following text that includes the main points and any important details.
                      Aim for a summary length of approximately 1500 words
                      Title - FP Country Execution Plan - NIGERIA_Strategy Execution Plan_10012014
Abstract - An execution plan for how the Gates Foundation plans to work in the Nigeria to contribute to the FP2020 goal of 120 million women by 2020
Summary - The image is a presentation slide from the Bill & Melinda Gates Foundation, specifically related to Nigeria's Family Planning Strategy Execution Plan for the fourth quarter of 2014. The slide is minimalistic, with a white background and a maroon header that includes the foundation's logo and name. Below this, in

c:\Users\Naga.Avuthu\Downloads\Testing_Share\.conda\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm




> Entering new LLMChain chain...
Prompt after formatting:

                      Write a comprehensive summary of the following text delimited by triple backquotes.
                      Aim for a summary length of approximately 800 words with out missing the important information the text.
                      ```The document appears to be a detailed execution plan for the Bill & Melinda Gates Foundation's work in Nigeria, aiming to contribute to the FP2020 goal of 120 million women using family planning methods by 2020.

The plan is structured into various sections, including context, Nigeria fact base, investment choices, and appendices. The content provides a comprehensive overview of the foundation's strategy, highlighting key areas such as:

1. Political commitment: Building support from Nigerian government officials and stakeholders to prioritize family planning.
2. Stakeholder alignment: Collaborating with partners, including the Federal Ministry of Health, to strengthen pol

In [24]:
summarized_docs

{'FP Country Execution Plan - NIGERIA_Strategy Execution Plan_10012014': 'The Bill & Melinda Gates Foundation has developed a comprehensive execution plan to contribute to the Family Planning 2020 (FP2020) goal of 120 million women using family planning methods by 2020 in Nigeria. The plan is structured into various sections, including context, Nigeria fact base, investment choices, and appendices, providing a detailed overview of the foundation\'s strategy to achieve this goal.\n\nThe plan highlights key areas of focus, including building political commitment from Nigerian government officials and stakeholders to prioritize family planning, collaborating with partners to strengthen policies and management systems, developing a costed implementation plan (CIP), and providing technical assistance to support policy rollout. Additionally, the plan emphasizes the importance of improving supply chain management, generating demand for family planning services, and tracking progress through t

In [25]:
with open ("summarized_json_new1.json", 'w') as f:
    json.dump(summarized_docs, f, indent =2)